Работа с языковыми моделями﻿

In [1]:
# Используйте эту ячейку, чтобы задать ключ в блокноте
import getpass
import os
from GIGACHAT_CREDENTIALS import TAVILY_API_KEY

os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY

In [2]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("Узнай на weatherapi погоду в Москве")
print(search_results)
# Если нужно, вы можете создать другие инструменты.
# После создания всех необходимых инструментов
# их можно сохранить в списке, к которому можно обращаться позднее.
tools = [search]

[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Москва', 'region': 'Moscow City', 'country': 'Россия', 'lat': 55.7522, 'lon': 37.6156, 'tz_id': 'Europe/Moscow', 'localtime_epoch': 1741079164, 'localtime': '2025-03-04 12:06'}, 'current': {'last_updated_epoch': 1741078800, 'last_updated': '2025-03-04 12:00', 'temp_c': 2.3, 'temp_f': 36.1, 'is_day': 1, 'condition': {'text': 'Light rain', 'icon': '//cdn.weatherapi.com/weather/64x64/day/296.png', 'code': 1183}, 'wind_mph': 9.4, 'wind_kph': 15.1, 'wind_degree': 259, 'wind_dir': 'W', 'pressure_mb': 1000.0, 'pressure_in': 29.53, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 93, 'cloud': 100, 'feelslike_c': -1.6, 'feelslike_f': 29.1, 'windchill_c': -2.7, 'windchill_f': 27.2, 'heatindex_c': 1.4, 'heatindex_f': 34.6, 'dewpoint_c': 0.5, 'dewpoint_f': 32.9, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 0.8, 'gust_mph': 14.1, 'gust_kph': 22.6}}"}, {'url': 'https://msk.nuipogoda.ru/4-%D0%BC%D0%B0%D1%80%D1%82%D0%B0-2025', 'conte

In [3]:
# | output: false
# | echo: false

from langchain_gigachat.chat_models import GigaChat
from GIGACHAT_CREDENTIALS import Scope

model = GigaChat(
    credentials=Scope,
    scope="GIGACHAT_API_PERS",
    model="GigaChat-Pro",
    verify_ssl_certs=False,
)

In [4]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="Привет!")])
response.content

'Привет! Что интересного у тебя на уме?'

In [5]:
model_with_tools = model.bind_tools(tools)

In [6]:
response = model_with_tools.invoke([HumanMessage(content="Привет!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Здравствуйте! Чем могу быть полезен?
ToolCalls: []


In [7]:
response = model_with_tools.invoke(
    [HumanMessage(content="Используй функцию поиска Tavily, чтобы сделать запрос на weatherapi и рассказать о погоде в Москве")]
)

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'погода в Москве'}, 'id': 'c92ac18e-e7c2-4e26-afea-3bc34cfa4472', 'type': 'tool_call'}]


In [10]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

In [11]:
response = agent_executor.invoke({"messages": [HumanMessage(content="Привет!")]})

response["messages"]

[HumanMessage(content='Привет!', additional_kwargs={}, response_metadata={}, id='fd872683-9232-446a-921a-df4cb620cd66'),
 AIMessage(content='Здравствуйте! Чем могу быть полезен?', additional_kwargs={'functions_state_id': 'b7672acc-b176-4522-b627-a7cba1122192'}, response_metadata={'token_usage': {'prompt_tokens': 108, 'completion_tokens': 17, 'total_tokens': 125}, 'model_name': 'GigaChat-Pro:1.0.26.20', 'x_headers': {'x-request-id': '1b50d9a1-4070-45e3-9d1a-f3c9442fb5bf', 'x-session-id': '8de7bdc7-d5de-4314-82f6-880e0eb80f8a', 'x-client-id': None}, 'finish_reason': 'stop'}, id='1b50d9a1-4070-45e3-9d1a-f3c9442fb5bf')]

In [12]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="Используй функцию поиска Tavily, чтобы сделать запрос на weatherapi и рассказать о погоде в Москве")]}
)
response["messages"]

[HumanMessage(content='Используй функцию поиска Tavily, чтобы сделать запрос на weatherapi и рассказать о погоде в Москве', additional_kwargs={}, response_metadata={}, id='7ad23452-9b8f-4a20-8814-083af87d15f7'),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': {'query': 'погода в Москве'}}, 'functions_state_id': '05ee1ced-c4d0-4845-b333-ea6d2ce5188e'}, response_metadata={'token_usage': {'prompt_tokens': 51, 'completion_tokens': 34, 'total_tokens': 85}, 'model_name': 'GigaChat-Pro:1.0.26.20', 'x_headers': {'x-request-id': '47c819e4-7b1c-44d0-8e39-7ac408c33851', 'x-session-id': 'bbdc40f9-a823-41dd-997e-db086c0d0ad7', 'x-client-id': None}, 'finish_reason': 'function_call'}, id='47c819e4-7b1c-44d0-8e39-7ac408c33851', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'погода в Москве'}, 'id': '2e5467f7-f768-4c5b-9103-216aa2a949e3', 'type': 'tool_call'}]),
 ToolMessage(content='[{"url": "https://www.gismeteo.ru/

In [13]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Используй функцию поиска Tavily, чтобы сделать запрос на weatherapi и рассказать о погоде в Москве")]}
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': {'query': 'погода в Москве'}}, 'functions_state_id': '0ed4a6d6-a476-4deb-880b-aece7f37727f'}, response_metadata={'token_usage': {'prompt_tokens': 51, 'completion_tokens': 34, 'total_tokens': 85}, 'model_name': 'GigaChat-Pro:1.0.26.20', 'x_headers': {'x-request-id': 'c509cd3d-be81-4782-8590-0498bf3f2455', 'x-session-id': '265aecbb-7621-4364-8c0e-ffa862d137c3', 'x-client-id': None}, 'finish_reason': 'function_call'}, id='c509cd3d-be81-4782-8590-0498bf3f2455', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'погода в Москве'}, 'id': 'fbc78098-707a-4d50-ad63-f4cf3bddcf98', 'type': 'tool_call'}])]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://www.gismeteo.ru/weather-moscow-4368/", "content": "GISMETEO: Погода в Москве сегодня, прогноз погоды Москва на сегодня, Москва (город федерального значения), Россия Герой РФ С

In [15]:
async for event in agent_executor.astream_events(
    {"messages": [HumanMessage(content="Используй функцию поиска Tavily, чтобы сделать запрос на weatherapi и рассказать о погоде в Москве")]},
    version="v1",
):
    kind = event["event"]
    if kind == "on_chain_start":
        if (
            event["name"] == "Agent"
        ):  # Назначается при создании агента с помощью метода `.with_config({"run_name": "Agent"})`
            print(
                f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
            )
    elif kind == "on_chain_end":
        if (
            event["name"] == "Agent"
        ):  # Назначается при создании агента с помощью метода `.with_config({"run_name": "Agent"})`
            print()
            print("--")
            print(
                f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
            )
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            print(content, end="|")
    elif kind == "on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")

--
Starting tool: tavily_search_results_json with inputs: {'query': 'погода в Москве'}
Done tool: tavily_search_results_json
Tool output was: content='[{"url": "https://www.gismeteo.ru/weather-moscow-4368/", "content": "GISMETEO: Погода в Москве сегодня, прогноз погоды Москва на сегодня, Москва (город федерального значения), Россия Герой РФ Сураев рассказал, как проявил себя Борисов в РоскосмосеПриказавшего сбить самолет над Кубанью приговорили к пожизненномуРоссиянам назвали продукты для борьбы с дефицитом витаминовВ Госдуме предложили добавить два новых выходных дняАлиев выделил $1 млн УкраинеСердечная недостаточность: как распознать и что делатьЗеленский заявил о готовности заключить сделку с ТрампомЭкс-президент Бразилии Болсонару намерен вывести страну из БРИКСКрым переходит к маловодному периоду - гидрологиЧем можно и нельзя запивать лекарстваМИД России пригрозил Франции ответом за эскалацию в вопросе СМИЗять Пригожина пропал на СВОБелый дом: Украина должна возместить расходы на 

In [16]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [17]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [18]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Привет! Меня зовут Вася")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Привет, Вася! Чем могу быть полезен?', additional_kwargs={'functions_state_id': '20fd1b86-d1ce-4191-8baf-796964aa04d8'}, response_metadata={'token_usage': {'prompt_tokens': 39, 'completion_tokens': 17, 'total_tokens': 56}, 'model_name': 'GigaChat-Pro:1.0.26.20', 'x_headers': {'x-request-id': 'b520f945-31a7-4f75-bdab-1476047b38c3', 'x-session-id': 'aa288ee8-366a-44e9-bcac-2d3c8a77249c', 'x-client-id': None}, 'finish_reason': 'stop'}, id='b520f945-31a7-4f75-bdab-1476047b38c3')]}}
----


In [19]:
config = {"configurable": {"thread_id": "xyz123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Как меня зовут?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='К сожалению, у меня нет доступа к такой информации о тебе. Я могу помогать с вопросами и задачами, но не знаю, как тебя зовут.', additional_kwargs={'functions_state_id': '7a72aff2-80ad-4c26-93b6-403f43e55d9c'}, response_metadata={'token_usage': {'prompt_tokens': 111, 'completion_tokens': 38, 'total_tokens': 149}, 'model_name': 'GigaChat-Pro:1.0.26.20', 'x_headers': {'x-request-id': 'e920cccd-43ae-442f-9433-f0769afb0ffa', 'x-session-id': 'ea79f02d-2724-4a00-8ae8-f6cd64022e69', 'x-client-id': None}, 'finish_reason': 'stop'}, id='e920cccd-43ae-442f-9433-f0769afb0ffa')]}}
----
